In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cProfile, pstats

import sys
sys.path.insert(0, '..')
%load_ext autoreload
%autoreload 2
from levitate import array_model
array_model.logger.setLevel(array_model.logging.WARNING)
array_model.logger.addHandler(array_model.logging.FileHandler('Optim.log', mode='w'))
array_model.logger.propagate = False

In [ ]:
loc_1 = np.array([-50e-3, 0, 0.2])
loc_2 = np.array([50e-3, 0, 0.2])

array = array_model.transducer_array()
optim = array_model.optimizer(array)

array.use_directivity = 'j1'
#array.phases = array.focus_phases()

gorkov_1 = array_model.gorkov_laplacian(array, loc_1)
gorkov_1.gradient_weights = (1, 1, 1)
gorkov_1.pressure_weight = 1
optim.add_objective(gorkov_1, 1)

gorkov_2 = array_model.gorkov_laplacian(array, loc_2)
gorkov_1.gradient_weights = (1, 1, 1)
gorkov_1.pressure_weight = 1
# optim.add_objective(gorkov_2, 1)

point = array_model.pressure_point(array, loc_2)
point.order = None
point.radius = 10e-3
point.gradient_weights = (1, 1, 1)
optim.add_objective(point, 1e-3)
# optim.add_objective(array_model.AmplitudeLimiting(array), 1e-3)

starting_phase = array.focus_phases(gorkov_1.focus) + array.vortex_signature()
starting_phase += np.random.uniform(-np.pi, np.pi, 256) * 0.01

In [ ]:
array.phases = np.mod(starting_phase + np.pi, 2 * np.pi) - np.pi
array.amplitudes = np.ones(array.num_transducers)
optim.basinhopping = True
optim.variable_amplitudes = False
pr = cProfile.Profile()
pr.enable()
optim()
pr.disable()

In [ ]:
array.phases = np.mod(optim.phases + np.pi,2*np.pi)-np.pi
array.amplitudes = optim.amplitudes
# array.phases = starting_phase; array.amplitudes = np.ones(256)
plotting_loc = loc_1

x,y,z = np.meshgrid(
    np.linspace(-0.1,0.1,201), 
    np.linspace(-0.1,0.1,201), 
    np.linspace(0,0.3,100), indexing='ij')

xidx = np.argmin(np.abs(x[:,0,0] - plotting_loc[0]))
yidx = np.argmin(np.abs(y[0,:,0] - plotting_loc[1]))
zidx = np.argmin(np.abs(z[0,0,:] - plotting_loc[2]))


pressures_yz = np.squeeze(array.calculate_pressure((x[xidx,:,:], y[xidx,:,:], z[xidx,:,:])))
pressures_xz = np.squeeze(array.calculate_pressure((x[:,yidx,:], y[:,yidx,:], z[:,yidx,:])))
pressures_xy = np.squeeze(array.calculate_pressure((x[:,:,zidx], y[:,:,zidx], z[:,:,zidx])))


fig = plt.figure()
plt.pcolormesh(x[:,:,zidx], y[:,:,zidx], np.abs(pressures_xy))
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
for obj, weight in optim.objective_list:
    if hasattr(obj, 'laplacian_axis'):
        plt.scatter(obj.focus[0], obj.focus[1],color='red')
    elif hasattr(obj, 'extra_points'):
        plt.scatter(obj.focus[0], obj.focus[1],color='green')
        for p in obj.extra_points:
            plt.scatter(p.focus[0], p.focus[1],color='blue')
    elif hasattr(obj, 'focus'):
        plt.scatter(obj.focus[0], obj.focus[1],color='green')
fig.show()

fig = plt.figure()
plt.pcolormesh(x[:,yidx,:], z[:,yidx,:], np.abs(pressures_xz))
plt.xlabel('x')
plt.ylabel('z')
plt.clim((np.abs(pressures_xy).min(), np.abs(pressures_xy).max()))
plt.colorbar()
for obj, weight in optim.objective_list:
    if hasattr(obj, 'laplacian_axis'):
        plt.scatter(obj.focus[0], obj.focus[2],color='red')
    elif hasattr(obj, 'extra_points'):
        plt.scatter(obj.focus[0], obj.focus[2],color='green')
        for p in obj.extra_points:
            plt.scatter(p.focus[0], p.focus[2],color='blue')
    elif hasattr(obj, 'focus'):
        plt.scatter(obj.focus[0], obj.focus[2],color='green')
fig.show()

fig = plt.figure()
plt.pcolormesh(y[xidx,:,:], z[xidx,:,:], np.abs(pressures_yz))
plt.xlabel('y')
plt.ylabel('z')
plt.clim((np.abs(pressures_xy).min(), np.abs(pressures_xy).max()))
plt.colorbar()
for obj, weight in optim.objective_list:
    if hasattr(obj, 'laplacian_axis'):
        plt.scatter(obj.focus[1], obj.focus[2],color='red')
    elif hasattr(obj, 'extra_points'):
        plt.scatter(obj.focus[1], obj.focus[2],color='green')
        for p in obj.extra_points:
            plt.scatter(p.focus[1], p.focus[2],color='blue')
    elif hasattr(obj, 'focus'):
        plt.scatter(obj.focus[1], obj.focus[2],color='green')
fig.show()

fig = plt.figure()
plt.scatter(array.transducer_positions[:,0], array.transducer_positions[:,1], c=array.phases/np.pi, s=144)
plt.colorbar()
fig.show()

fig = plt.figure()
plt.scatter(array.transducer_positions[:,0], array.transducer_positions[:,1], c=array.signature(focus=plotting_loc)/np.pi, s=144)
plt.colorbar()
fig.show()

fig = plt.figure()
plt.scatter(array.transducer_positions[:,0], array.transducer_positions[:,1], c=array.amplitudes, s=144)
plt.colorbar()
fig.show()
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111)
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)

phase_list = optim.result.allvecs
#phase_list.insert(0, starting_phase)
nphases = len(phase_list)
nplot = nphases
phase_matrix = np.empty((nphases, array.num_transducers))
jac_matrix = np.empty((nphases, array.num_transducers))
gnorm = np.empty(nphases)
fval = np.empty(nphases)

for idx, phase in enumerate(phase_list):
    phase_matrix[idx,:] = phase
    jac_matrix[idx,:] = optim.jacobian(phase)
    fval[idx] = optim.function(phase)
    gnorm[idx] = array_model.norm(jac_matrix[idx,:], np.inf)
for trans in range(array.num_transducers):
    phase = phase_matrix[:nplot,trans]
    jac = jac_matrix[:nplot,trans]
    #ax.plot(trans*np.ones(len(phase)), np.arange(len(phase)), phase/np.pi)
    #ax2.plot(trans*np.ones(len(phase)), np.arange(len(phase)), jac)
ax.imshow(phase_matrix.T)
ax2.imshow(np.log(np.abs(jac_matrix.T)))
    
fig.show()
fig2.show()

fig3 = plt.figure()
plt.semilogy(gnorm)
fig4 = plt.figure()
plt.semilogy(fval)
plt.semilogy(-fval)
fig3.show()
fig4.show()